In [59]:
# **************************** IMPORTANT ****************************
'''
This cell configuration settings for the Notebook. 
You can run one role at a time to evaluate the performance of the model
Change the variable names to run for multiple roles

In this model - cosine distance is calculated between the skills and
the course description. Each skill has a weighted score based on the 
popularity of the skill. This is derived by endorsements of the respective
skill by other linkedin connections.
'''


# *******************************************************************
# For each role a list of category names are grouped. 
# Please don't change these variables

label_data_scientist = ['Data Science','Machine Learning',
                           'Data Analysis', 'Business Intelligence',
                           'Data Mining','Data Visualization']

label_software_engineer = ['Software Development','Computer Science',
                           'Programming Languages', 'Software Development',
                           'Web Development','Algorithms and Data Structures']
                                       
# *******************************************************************


# *******************************************************************
# Environment and Config Variables. Change these variables as required.

my_fpath_courses = "../Data/main_coursera.csv"

my_fpath_skills_DataScientist = "../Data/Word2Vec-Google/Word2VecGoogle_DataScientist.csv"

my_fpath_skills_SoftwareEngineer = "../Data/Word2Vec-Google/Word2VecGoogle_SoftwareEngineer.csv" 
                                       
# *******************************************************************


In [60]:
# Importing required modules/packages

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk, string


In [61]:
# Downloading the stopwords like i, me, and, is, the etc.

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/DV/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [62]:
# Loading courses and skills data from the CSV files

df_courses = pd.read_csv(my_fpath_courses)

df_DataScientist = pd.read_csv(my_fpath_skills_DataScientist)
df_DataScientist = df_DataScientist.drop('Role', 1)
df_DataScientist.columns = ['Course Id', 'DataScientist_Skill_Score', 'DataScientist_Role_Score']

df_SoftwareEngineer = pd.read_csv(my_fpath_skills_SoftwareEngineer)
df_SoftwareEngineer = df_SoftwareEngineer.drop('Role', 1)
df_SoftwareEngineer.columns = ['Course Id','SoftwareEngineer_Skill_Score', 'SoftwareEngineer_Role_Score']


In [63]:
# Merging the csv files

df_cosdist = df_DataScientist.merge(df_SoftwareEngineer, on = 'Course Id', how = 'outer')


In [64]:
# Exploring data dimensionality, feature names, and feature types.

print(df_courses.shape,"\n")

print(df_cosdist.shape,"\n")

print(df_courses.columns, "\n")

print(df_cosdist.shape,"\n")

print(df_courses.describe(), "\n")

print(df_cosdist.describe(), "\n")


(2213, 18) 

(2213, 5) 

Index(['Unnamed: 0', 'Course Id', 'Course Name', 'Course Description', 'Slug',
       'Provider', 'Universities/Institutions', 'Parent Subject',
       'Child Subject', 'Category', 'Url', 'Length', 'Language',
       'Credential Name', 'Rating', 'Number of Ratings', 'Certificate',
       'Workload'],
      dtype='object') 

(2213, 5) 

        Unnamed: 0    Course Id      Length       Rating  Number of Ratings  \
count  2213.000000  2213.000000  964.000000  2213.000000        2213.000000   
mean   1430.685043  4816.998192    6.063278     2.352785          10.321735   
std     887.770407  3033.878865    2.724669     2.129134         110.680382   
min       0.000000   303.000000    1.000000     0.000000           0.000000   
25%     631.000000  1829.000000    4.000000     0.000000           0.000000   
50%    1455.000000  4880.000000    6.000000     3.000000           1.000000   
75%    2216.000000  7329.000000    7.000000     4.428571           4.000000   
max  

In [65]:
# Quick check to see if the dataframe showing the right results

df_cosdist.head(20)

,Course Id,DataScientist_Skill_Score,DataScientist_Role_Score,SoftwareEngineer_Skill_Score,SoftwareEngineer_Role_Score
0,303,0.250289,0.268095,0.249944,0.135970
1,305,0.237911,0.205215,0.328530,0.437830
2,306,0.191053,0.124907,0.212947,0.069229
3,307,0.271128,0.146402,0.271968,0.050458
4,308,0.239865,0.126424,0.267256,0.119867
5,309,0.267912,0.283366,0.268586,0.186795
6,316,0.235680,0.304023,0.231349,0.106512
7,317,0.212704,0.143987,0.224335,0.064702
8,318,0.207079,0.170354,0.247371,0.223502
9,322,0.233237,0.128654,0.210427,0.151994


In [66]:
# Joining two dataframes - Courses and the Cosein Similarity Results based on the 'Course Id' variable. 
# Inner joins: Joins two tables with the common rows. This is a set operateion.

df_courses_score = df_courses.merge(df_cosdist, on ='Course Id', how='inner')

In [71]:
df_courses_score.sort_values(by=['DataScientist_Role_Score'], ascending=False).head(40)

,Unnamed: 0,Course Id,Course Name,Course Description,Slug,Provider,Universities/Institutions,Parent Subject,Child Subject,Category,...,Language,Credential Name,Rating,Number of Ratings,Certificate,Workload,DataScientist_Skill_Score,DataScientist_Role_Score,SoftwareEngineer_Skill_Score,SoftwareEngineer_Role_Score
506,579,1712,The Data Scientist’s Toolbox,In this course you will get an introduction to...,coursera-the-data-scientist-s-toolbox,Coursera,Johns Hopkins University,Data Science,NaN,Data Science,...,English,Data Science,3.263804,163,1.0,1-4 hours a week,0.252362,0.570030,0.247776,0.216195
836,1004,3478,Statistics for Genomic Data Science,An introduction to the statistics behind the m...,coursera-statistics-for-genomic-data-science,Coursera,Johns Hopkins University,Data Science,Bioinformatics,Bioinformatics,...,English,Genomic Data Science,1.666667,3,1.0,5-7 hours a week,0.236669,0.506991,0.223095,0.109433
833,1001,3475,Bioconductor for Genomic Data Science,Learn to use tools from the Bioconductor proje...,coursera-bioconductor-for-genomic-data-science,Coursera,Johns Hopkins University,Data Science,Bioinformatics,Bioinformatics,...,English,Genomic Data Science,3.000000,4,1.0,3-5 hours a week,0.247400,0.506566,0.249422,0.118902
109,110,451,Introduction to Data Science,Join the data revolution. Companies are search...,coursera-introduction-to-data-science,Coursera,University of Washington,Data Science,NaN,Data Science,...,English,Data Science at Scale,3.566667,30,1.0,10-12 hours a week,0.236196,0.496296,0.210213,0.124342
2212,2953,9969,On Being a Scientist,On Being a Scientist will provide learners wi...,coursera-on-being-a-scientist,Coursera,Leiden University,Science,NaN,Science,...,English,NaN,0.000000,0,1.0,NaN,0.169751,0.484270,0.146246,0.221711
831,999,3473,Genomic Data Science with Galaxy,Learn to use the tools that are available from...,coursera-genomic-data-science-with-galaxy,Coursera,Johns Hopkins University,Data Science,Bioinformatics,Bioinformatics,...,English,Genomic Data Science,1.750000,12,1.0,3-5 hours a week,0.233570,0.483362,0.233797,0.112722
58,58,388,Computing for Data Analysis,This course is about learning the fundamental ...,coursera-computing-for-data-analysis,Coursera,Johns Hopkins University,Data Science,Data Analysis,Data Analysis,...,English,Data Science,3.407407,27,1.0,3-5 hours a week,0.273819,0.468590,0.257776,0.236610
687,805,2737,Data Visualization,Learn the general concepts of data mining alon...,coursera-data-visualization,Coursera,University of Illinois at Urbana-Champaign,Data Science,Data Visualization,Data Visualization,...,English,Data Mining,3.263158,19,1.0,4-6 hours a week,0.252448,0.467397,0.233035,0.244885
1118,1471,5028,Communicating Data Science Results,Important note: The second assignment in this ...,coursera-communicating-data-science-results,Coursera,University of Washington,Data Science,NaN,Data Science,...,English,NaN,1.000000,2,1.0,NaN,0.260046,0.467385,0.263126,0.126902
1444,1934,6534,Materials Data Sciences and Informatics,This course aims to provide a succinct overvie...,coursera-materials-data-sciences-and-informatics,Coursera,Georgia Institute of Technology,Engineering,NaN,Engineering,...,English,NaN,0.000000,0,1.0,NaN,0.234480,0.464033,0.221896,0.251807


In [38]:
# Tranforming and shaping the data to create the confusion matrix for the ROLE: DATA SCIENTIST

my_DataScientist_skill_score = 'DataScientist_Role_Score'
my_DataScientist_final_score = 'DataScientist_Final_Score'
y_actu_DataScientist         = ''
y_pred_DataScientist         = ''

df_courses_score[[my_DataScientist_final_score]] = df_courses_score[[my_DataScientist_skill_score]]

df_courses_score['DataScientist_Predict'] = (df_courses_score[[my_DataScientist_final_score]] >= 0.5)

df_courses_score['DataScientist_Label'] = df_courses_score.Category.isin(label_data_scientist)

y_pred_DataScientist = pd.Series(df_courses_score['DataScientist_Predict'], name='Predicted')

y_actu_DataScientist = pd.Series(df_courses_score['DataScientist_Label'], name='Actual')

df_confusion_DataScientist = pd.crosstab(y_actu_DataScientist, y_pred_DataScientist , rownames=['Actual'], colnames=['Predicted'], margins=False)


In [39]:
# Tranforming and shaping the data to create the confusion matrix for the ROLE: SOFTWARE ENGINEER

my_SoftwareEngineer_skill_score = 'SoftwareEngineer_Role_Score'
my_SoftwareEngineer_final_score = 'SoftwareEngineer_Final_Score'
y_actu_SoftwareEngineer         = ''
y_pred_SoftwareEngineer         = ''

df_courses_score[[my_SoftwareEngineer_final_score]] = df_courses_score[[my_SoftwareEngineer_skill_score]]

df_courses_score['SoftwareEngineer_Predict'] = (df_courses_score[[my_SoftwareEngineer_final_score]] >= 0.5)

df_courses_score['SoftwareEngineer_Label'] = df_courses_score.Category.isin(label_software_engineer)

y_pred_SoftwareEngineer = pd.Series(df_courses_score['SoftwareEngineer_Predict'], name='Predicted')

y_actu_SoftwareEngineer = pd.Series(df_courses_score['SoftwareEngineer_Label'], name='Actual')

df_confusion_SoftwareEngineer = pd.crosstab(y_actu_SoftwareEngineer, y_pred_SoftwareEngineer , rownames=['Actual'], colnames=['Predicted'], margins=False)


In [40]:
df_confusion_DataScientist


Predicted,False,True
Actual,,
False,2085,2
True,125,1


In [41]:
df_confusion_SoftwareEngineer

Predicted,False,True
Actual,,
False,2064,3
True,146,0


In [42]:
# Performance summary for the ROLE: DATA SCIENTIST


try:
    tn_DataScientist = df_confusion_DataScientist.iloc[0][False]
except:
    tn_DataScientist = 0
    
try:
    tp_DataScientist =  df_confusion_DataScientist.iloc[1][True]
except:
    tp_DataScientist = 0

    
try:
    fn_DataScientist = df_confusion_DataScientist.iloc[1][False]
except:
    fn_DataScientist = 0
    
try:
    fp_DataScientist =  df_confusion_DataScientist.iloc[0][True]
except:
    fp_DataScientist = 0  
    
    
total_count_DataScientist = tn_DataScientist + tp_DataScientist + fn_DataScientist + fp_DataScientist


print('Data Scientist Accuracy Rate : ', (tn_DataScientist + tp_DataScientist) / total_count_DataScientist)

print('Data Scientist Misclassifcation Rate : ', (fn_DataScientist + fp_DataScientist) / total_count_DataScientist)

print('Data Scientist True Positive Rate : ', tp_DataScientist / (tp_DataScientist + fn_DataScientist))

print('Data Scientist False Positive Rate : ', fp_DataScientist / (tn_DataScientist + fp_DataScientist))


Data Scientist Accuracy Rate :  0.9426118391323994
Data Scientist Misclassifcation Rate :  0.05738816086760054
Data Scientist True Positive Rate :  0.007936507936507936
Data Scientist False Positive Rate :  0.0009583133684714902


In [43]:
# Performance summary for the ROLE: SOFTWARE ENGINEER


try:
    tn_SoftwareEngineer = df_confusion_SoftwareEngineer.iloc[0][False]
except:
    tn_SoftwareEngineer = 0
    
try:
    tp_SoftwareEngineer =  df_confusion_SoftwareEngineer.iloc[1][True]
except:
    tp_SoftwareEngineer = 0

    
try:
    fn_SoftwareEngineer = df_confusion_SoftwareEngineer.iloc[1][False]
except:
    fn_SoftwareEngineer = 0
    
try:
    fp_SoftwareEngineer =  df_confusion_SoftwareEngineer.iloc[0][True]
except:
    fp_SoftwareEngineer = 0  
    
    
total_count_SoftwareEngineer = tn_SoftwareEngineer + tp_SoftwareEngineer + fn_SoftwareEngineer + fp_SoftwareEngineer


print('Software Engineer Accuracy Rate : ', (tn_SoftwareEngineer + tp_SoftwareEngineer) / total_count_SoftwareEngineer)

print('Software Engineer Misclassifcation Rate : ', (fn_SoftwareEngineer + fp_SoftwareEngineer) / total_count_SoftwareEngineer)

print('Software Engineer True Positive Rate : ', tp_SoftwareEngineer / (tp_SoftwareEngineer + fn_SoftwareEngineer))

print('Software Engineer False Positive Rate : ', fp_SoftwareEngineer / (tn_SoftwareEngineer + fp_SoftwareEngineer))


Software Engineer Accuracy Rate :  0.9326705829191143
Software Engineer Misclassifcation Rate :  0.06732941708088568
Software Engineer True Positive Rate :  0.0
Software Engineer False Positive Rate :  0.001451378809869376
